# ECIES

## Сетап

$E: y^2 = x^3 + a x + b \pmod{p}$

$A$ - случайная точка, длп сложный

$s$ - секрет

$B = sA$

$(E, A, B, p)$ - публичное

$s$ - приватное

## Шифрование

$k$ - случайное

$R = k * A$

$Z = k * B$

$k_1k_2 = HASH(R||Z)$

$k_1 = k_1k_2[:16]$

$k_2 = k_1k_2[16:]$

$c = AES(k_11, m)$

$t = HASH(c||k_2)$

$(R, c, t)$

## Расшифрование

$Z = s R$

$k_1k_2 = HASH(R||Z)$

$k_1 = k_1k_2[:16]$

$k_2 = k_1k_2[16:]$

$t_1 = HASH(c||k_2)$

$t_1 == t$ ? 

$m = AES(k1, c)$

In [6]:
from os import urandom

from Crypto.Cipher import AES
from Crypto.Hash import BLAKE2b
from Crypto.Util.number import getPrime
from Crypto.Util.Padding import pad, unpad
from sage.all import GF, EllipticCurve, randint


def setup(nbit):
    p = getPrime(nbit)
    a, b = 1, 2
    E = EllipticCurve(GF(p), [a, b])
    N = E.order()
    A = E.random_point()
    s = randint(1, N - 1)
    B = s * A
    return (E, A, B, p), s


def encrypt(pkey, msg):
    E, A, B, p = pkey
    N = E.order()
    k = randint(1, N - 1)
    R = k * A
    Z = k * B

    H1 = BLAKE2b.new(digest_bytes=32)
    H1.update(f"{R.xy()}||{Z.xy()}".encode())
    k1k2 = H1.digest()

    k1, k2 = k1k2[:16], k1k2[16:]

    iv = urandom(16)
    C = AES.new(k1, mode=AES.MODE_CBC, iv=iv)
    ct = iv + C.encrypt(pad(msg, 16))

    H2 = BLAKE2b.new(digest_bytes=16)
    H2.update(ct + b"||" + k2)
    t = H2.digest()
    return (R, ct, t)


def decrypt(pkey, skey, C):
    R, ct, t = C
    E, A, B, p = pkey

    Z = skey * R
    H1 = BLAKE2b.new(digest_bytes=32)
    H1.update(f"{R.xy()}||{Z.xy()}".encode())
    k1k2 = H1.digest()

    k1, k2 = k1k2[:16], k1k2[16:]

    H2 = BLAKE2b.new(digest_bytes=16)
    H2.update(ct + b"||" + k2)
    t_ = H2.digest()
    if t_ != t:
        return None

    iv, ct = ct[:16], ct[16:]
    D = AES.new(k1, mode=AES.MODE_CBC, iv=iv)
    msg = unpad(D.decrypt(ct), 16)
    return msg


pkey, skey = setup(128)
C = encrypt(pkey, b"Hey Bubs")
M = decrypt(pkey, skey, C)
print(M)

b'Hey Bubs'


# Немножечко про ZKP

В какой то момент мы потеряли человечность и начали спаривать точки на кривой.

Пейринги или спаривания помогают сделать следующее:

Возьмем две точки из $E(F_p)[r]$ 

Тогда $e(P, Q): E(F_p)[r] \times E(F_p)[r] \to F_r:{*}$ - билинейное спаривание

Билинейное оно, потому что:

$e(P_1 + P_2, Q) = e(P_1, Q) * e(P_2, Q)$

$e(P, Q_1 + Q_2) = e(P, Q_1) * e(P, Q_2)$

Из этого вытекает, что $e(n * P, Q) = e(P, Q)^n$ А это очень интересно, так как мы не зная $n$ можем его вынимать и как то с ним взаимодействовать. Хотя мы все еще его не знаем!


Это свойство и было полезно в ZKP, так как часто нам нужно посчитать произведение каких то двух многочленов, которые мы не должны знать.

Для этого используют следующее:

Берут случайное число $\tau$, которое сразу же забывают после генерации точек: 

$G, \tau * G, \tau^2 * G, ..., \tau^n * G$

Теперь комбинируя эти точки мы можем получить любой многочлен($\tau$ здесь играет роль неизвестного x)

$P = 2 * G_0 + 4 * G_1 - G_2$

$Q = G_0 - 4 * G_1$

$R = 2 * G_0 - 4 * G_1 - 17 * G_2 + 4 * G_3$